### Model Evaluation

Here we evaluate the four models using the test images



In [1]:
# organize imports
from __future__ import print_function

from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
import numpy as np
import h5py
import os
import json
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb

In [2]:
# load the user configs
with open('room_recognition/conf/conf_eval.json') as f:    
  config = json.load(f)

# config variables

seed      = config["seed"]
train_features_path   = config["train_features_path"]
train_labels_path   = config["train_labels_path"]
test_features_path   = config["test_features_path"]
test_labels_path   = config["test_labels_path"]
results_log     = config["results_logistic"]
classifier_path_log = config["classifier_path_logistic"]
model_log=config["model_path_logistic"]
train_path    = config["train_path"]
num_classes   = config["num_classes"]
results_svm     = config["results_svm"]
classifier_path_svm = config["classifier_path_svm"]
model_svm=config["model_path_svm"]
results_xgboost    = config["results_xgboost"]
classifier_path_xgboost = config["classifier_path_xgboost"]
model_xgboost=config["model_path_xgboost"]
results_rf    = config["results_rf"]
classifier_path_rf = config["classifier_path_rf"]
model_rf=config["model_path_rf"]

In [3]:
# import train features and labels
h5f_data_train  = h5py.File(train_features_path, 'r')
h5f_label_train = h5py.File(train_labels_path, 'r')

train_features_string = h5f_data_train['dataset_1']
train_labels_string   = h5f_label_train['dataset_1']

trainfeatures = np.array(train_features_string)
trainlabels   = np.array(train_labels_string)

h5f_data_train.close()
h5f_label_train.close()

In [4]:
# verify the shape of features and labels
print ("[INFO] features shape: {}".format(trainfeatures.shape))
print ("[INFO] labels shape: {}".format(trainlabels.shape))


[INFO] features shape: (1040, 2048)
[INFO] labels shape: (1040,)


In [5]:
#Import test features
h5f_data_test  = h5py.File(test_features_path, 'r')
h5f_label_test = h5py.File(test_labels_path, 'r')

test_features_string = h5f_data_test['dataset_1']
test_labels_string   = h5f_label_test['dataset_1']

testfeatures = np.array(test_features_string)
testlabels   = np.array(test_labels_string)

h5f_data_test.close()
h5f_label_test.close()

In [6]:
# verify the shape of features and labels
print ("[INFO] features shape: {}".format(testfeatures.shape))
print ("[INFO] labels shape: {}".format(testlabels.shape))

[INFO] features shape: (160, 2048)
[INFO] labels shape: (160,)


In [7]:
def print_report( clf_name, y, y_pred):
    ## This function  takes a model name(clf_name), y labels(y), predicted y labels(y_pred) and predicted probabilities(y_score)
    ## It calculates  precision, recall and F1 score, confusion matrix 
    ## It returns a report with the metrics and the confusion matrix
    
    clf_rep = metrics.precision_recall_fscore_support(y, y_pred)
    
    out_dict = {     
                    "precision_score" :clf_rep[0].round(2),
                     "recall_score" : clf_rep[1].round(2),
                     "f1_score" : clf_rep[2].round(2),
                      "support" : clf_rep[3]
                     
        
                }
    out_df = pd.DataFrame(out_dict)
    avg_tot = (out_df.apply(lambda x: round(x.mean(), 2) if x.name!="support" else  round(x.sum(), 2)).to_frame().T)
    avg_tot.index = ["avg/total"]
    
    out_df = out_df.append(avg_tot)

    cols = [ 'precision_score', 'recall_score','f1_score', "support"]
    models_report = pd.DataFrame(columns=cols)
   
   
    models_report = models_report.append(out_df)
    models_report["model"]= clf_name
    
    conf_matrix=dict()
    
    conf_matrix = pd.crosstab(y, y_pred, rownames=['True'], colnames= ['Predicted'], margins=False)
        

    
    return models_report, conf_matrix

In [8]:
def run_models(clfs_dict, test_X, test_y):
    ## This function takes a dictionary clfs_dict with classifier name and classsifier as keys and values, and X and y
    ## It fits the classifier
    ## returns predicted y and predicted probabilities for y 

    clfs = clfs_dict
    
    cols = ['model', 'precision_score', 'recall_score','f1_score', "support"]
    
    Full_report = pd.DataFrame(columns = cols)
   
    conf_matrix_all = dict()
    Full_report_dict=dict()

    for clf, clf_name in zip(clfs.values(), clfs.keys()):
        
        
        test_y_pred=clf.predict(test_X)
        
        report, conf_matrix = print_report(clf_name, test_y, test_y_pred)
    
        Full_report = Full_report.append(report, ignore_index = False)
    
        
        conf_matrix_all[clf_name] =conf_matrix
    
    return Full_report,  conf_matrix_all
    

In [9]:
clfs ={'LogisticRegression' : joblib.load(model_log),
      
       'RandomForestClassifier': joblib.load(model_rf),
       
        "Support Vector" : joblib.load(model_svm) 
          
  }
    

Full_report,  conf_matrix_all = run_models(clfs, testfeatures, testlabels)

###### Load XGBOOST Model

In [10]:
bst = xgb.Booster()
bst.load_model(model_xgboost)

In [11]:
dtest = xgb.DMatrix(testfeatures)
ypred = bst.predict(dtest)



In [12]:
report, conf_matrix = print_report("xgboost", testlabels, ypred)

###### GET EVALUATION METRICS FOR ALL 

In [13]:
Full_report = Full_report.append(report, ignore_index = False)
    
        
conf_matrix_all["xgboost"] =conf_matrix  

In [14]:

Full_report=Full_report.rename(index={0: 'bathroom', 1:'bedroom', 2: 'corridor', 3: 'dining_room', 4: 'kitchen', 5:'living room', 6: 'pantry', 7:"staircase"})
Full_report= Full_report[['model', 'precision_score', 'recall_score', 'f1_score', 'support']]


In [15]:
Full_report

,model,precision_score,recall_score,f1_score,support
bathroom,LogisticRegression,0.90,0.90,0.90,20.0
bedroom,LogisticRegression,0.83,0.75,0.79,20.0
corridor,LogisticRegression,1.00,1.00,1.00,20.0
dining_room,LogisticRegression,0.82,0.90,0.86,20.0
kitchen,LogisticRegression,0.94,0.75,0.83,20.0
living room,LogisticRegression,0.71,0.75,0.73,20.0
pantry,LogisticRegression,0.91,1.00,0.95,20.0
staircase,LogisticRegression,0.95,1.00,0.98,20.0
avg/total,LogisticRegression,0.88,0.88,0.88,160.0
bathroom,RandomForestClassifier,0.90,0.95,0.93,20.0


In [16]:
conf_matrix_all["LogisticRegression"]

Predicted,0,1,2,3,4,5,6,7
True,,,,,,,,
0,18,1,0,0,0,0,1,0
1,1,15,0,1,0,3,0,0
2,0,0,20,0,0,0,0,0
3,0,0,0,18,0,1,1,0
4,1,0,0,2,15,2,0,0
5,0,2,0,1,1,15,0,1
6,0,0,0,0,0,0,20,0
7,0,0,0,0,0,0,0,20


In [17]:
conf_matrix_all["RandomForestClassifier"]

Predicted,0,1,2,3,4,5,6,7
True,,,,,,,,
0,19,1,0,0,0,0,0,0
1,1,16,0,1,0,2,0,0
2,0,0,20,0,0,0,0,0
3,0,0,0,19,0,0,1,0
4,1,0,0,3,14,1,0,1
5,0,3,1,3,0,13,0,0
6,0,0,0,0,0,0,20,0
7,0,0,1,0,0,0,0,19


In [18]:
conf_matrix_all["Support Vector"]

Predicted,0,1,2,3,4,5,6,7
True,,,,,,,,
0,17,2,0,0,0,0,1,0
1,1,18,0,0,0,1,0,0
2,0,0,19,1,0,0,0,0
3,0,0,0,18,0,1,1,0
4,1,0,0,3,13,3,0,0
5,0,2,0,2,0,15,0,1
6,0,0,0,0,0,0,20,0
7,0,0,0,0,0,0,0,20


In [19]:
conf_matrix_all["xgboost"]

Predicted,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
True,,,,,,,,
0,16,2,0,0,1,1,0,0
1,1,16,0,0,0,3,0,0
2,0,0,19,1,0,0,0,0
3,0,0,0,19,0,0,1,0
4,1,0,0,3,14,1,0,1
5,0,3,0,3,1,13,0,0
6,0,0,0,0,0,0,20,0
7,0,0,1,0,0,0,0,19


In [20]:
with open('Full_report.txt', 'wb') as f:
  pickle.dump(Full_report, f)

In [21]:
with open('Confmatrix.txt', 'wb') as h:
 pickle.dump(conf_matrix_all, h)